In [1]:
from keras.models import model_from_json
from keras.preprocessing.image import array_to_img, img_to_array, load_img
import h5py
import os
from PIL import Image
from scipy.misc import imread, imresize, imsave
import numpy as np

print ('Mosaic: detect porngraghic')
print ('=====================================')

Using Theano backend.


Mosaic: detect porngraghic


In [2]:
from keras.applications import vgg16
from keras import backend as K
from keras.models import Model,Sequential
from keras.layers import Flatten, Dense, Input
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import decode_predictions, preprocess_input

In [3]:
vPorn = '/Users/holazhai/Downloads/NDPI/FramevPorn2/'
vNonPorn = '/Users/holazhai/Downloads/NDPI/FramevNonPorn/'

img_width = 224
img_height = 224
batch_size = 32
if K.image_dim_ordering() == 'th':
    img_size = (3, img_width, img_height)
else:
    img_size = (img_width, img_height, 3)
mosaic = Input(batch_shape=(None,) + img_size)
model = vgg16.VGG16(input_tensor=mosaic,weights='imagenet', include_top=False)
model.trainable=False

for l in model.layers:
    l.trainable=False
#model.summary()

   90112/58889096 [..............................] - ETA: 8344s

KeyboardInterrupt: 

In [4]:
def preprocess_image(image_path):
    try:
        img = load_img(image_path, target_size=(img_width, img_height))
    except:
        print 'error'
        return 'error'
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg16.preprocess_input(img)
    return img

# util function to convert a tensor into a valid image
def deprocess_image(x):
    if K.image_dim_ordering() == 'th':
        x = x.reshape((3, img_width, img_height))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((img_width, img_height, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [5]:
def preprocess_data(path, labs):
    data = []
    label = []
    filelist = os.listdir(path)
    for f in filelist:
        preimage = preprocess_image(path + f)
        if preimage <> 'error':
            #print type(preimage)
            data.extend(preimage)
            label.extend([labs])
    return data, label

In [6]:
datavPorn, labelvPorn = preprocess_data(vPorn,1)
datavNonPorn, labelvNonPorn = preprocess_data(vNonPorn,0)

data = datavPorn
data.extend(datavNonPorn)

label = labelvPorn
label.extend(labelvNonPorn)

data = np.array(data)
#测试
#data2 = data[:10,:,:,:]

/Users/holazhai/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


error


In [7]:
data.shape

(4300, 224, 224, 3)

In [8]:
final_model = Sequential()
final_model.add(model)
final_model.add(Flatten(name='flatten'))
final_model.add(Dense(2048, activation='relu', name='fc1'))
final_model.add(Dense(1024, activation='relu', name='fc2'))
final_model.add(Dense(1, activation='sigmoid', name='predictions'))

final_model.compile(optimizer='Adadelta', loss='binary_crossentropy', metrics=['accuracy'])
final_model.layers[0].trainable=False

In [9]:
#model.get_weights()[0][0][0]

array([[ 0.42947057,  0.11727387,  0.03401296,  0.35422093, -0.0865837 ,
         0.22968295,  0.06683242,  0.04133838,  0.13026784,  0.03291035,
         0.09615457, -0.09222532, -0.15587331,  0.0522968 , -0.08324417,
         0.15728769,  0.0291196 ,  0.05094835,  0.19055748, -0.04372229,
         0.36180311,  0.23902874,  0.16667192, -0.18046746,  0.04404821,
         0.09592837, -0.04381131, -0.18848351, -0.04399502, -0.18985446,
        -0.23386982,  0.12836744, -0.11059975,  0.00571336, -0.24479656,
        -0.30530283,  0.060938  , -0.32816252,  0.1711953 ,  0.10265407,
        -0.11063665,  0.33012986, -0.05869728,  0.18478717,  0.01050853,
         0.10668027, -0.13086924, -0.36283946,  0.06440807,  0.09335972,
         0.00401619,  0.37814805,  0.05268804, -0.27078936,  0.11060741,
        -0.11819526,  0.04298935,  0.35323623, -0.28468162, -0.20289008,
        -0.284944  , -0.13224158, -0.05334752,  0.00757738],
       [ 0.55037946,  0.02087744,  0.09883115,  0.43703237, -0.

In [10]:
final_model.fit(data, label, batch_size=16, nb_epoch=1,shuffle=True,verbose=1,validation_split=0.5)

Train on 2150 samples, validate on 2150 samples
Epoch 1/1
2150/2150 [==============================] - 4321s - loss: 2.5090 - acc: 0.8414 - val_loss: 15.9421 - val_acc: 0.0000e+00


In [11]:
final_model.save_weights('/mnt2/zhaihuixin/src/jianhuang.h5')

print ('Try to Predict input images:')
print ('=====================================')

re_img = preprocess_image('/mnt2/zhaihuixin/6.jpg')  # this is a jpg image (size (1, 64, 64, 3))
print model.predict(re_img)
print model.predict_proba(re_img)

path = '/mnt2/zhaihuixin/gol075/'
files = os.listdir(path) 
for f in files:
    x_img = preprocess_image(path+f)
    print (f + ' probability is: %3.3f%% ' %(100*model.predict_proba(x_img)))

array([[ 0.42947057,  0.11727387,  0.03401296,  0.35422093, -0.0865837 ,
         0.22968295,  0.06683242,  0.04133838,  0.13026784,  0.03291035,
         0.09615457, -0.09222532, -0.15587331,  0.0522968 , -0.08324417,
         0.15728769,  0.0291196 ,  0.05094835,  0.19055748, -0.04372229,
         0.36180311,  0.23902874,  0.16667192, -0.18046746,  0.04404821,
         0.09592837, -0.04381131, -0.18848351, -0.04399502, -0.18985446,
        -0.23386982,  0.12836744, -0.11059975,  0.00571336, -0.24479656,
        -0.30530283,  0.060938  , -0.32816252,  0.1711953 ,  0.10265407,
        -0.11063665,  0.33012986, -0.05869728,  0.18478717,  0.01050853,
         0.10668027, -0.13086924, -0.36283946,  0.06440807,  0.09335972,
         0.00401619,  0.37814805,  0.05268804, -0.27078936,  0.11060741,
        -0.11819526,  0.04298935,  0.35323623, -0.28468162, -0.20289008,
        -0.284944  , -0.13224158, -0.05334752,  0.00757738],
       [ 0.55037946,  0.02087744,  0.09883115,  0.43703237, -0.